In [ ]:
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

from feedback_grape.fgrape import evaluate_on_longer_time
from helpers import (
    init_fgrape_protocol,
    test_implementations,
)
from tqdm import tqdm
import json, jax, re
from library.utils.FgResult_to_dict import FgResult_to_dict
import numpy as np

test_implementations()

jax.config.update("jax_enable_x64", True)

# Physical parameters
gamma_p_global = None # None = Take value from filename
gamma_m_global = None # None = Take value from filename
evaluation_state_type = None # None = Take value from filename

# Evaluation parameters
evaluation_time_steps = 200 # Number of time steps for evaluation
batch_size = 128 # Number of random states to evaluate in parallel
N_parallel_threads = 8 # Number of parallel threads for training
dir_ = "./11_results/"

#accurate_pairs = [# Pairs of (gamma_m, gamma_p) to evaluate on larger batches (deactivated)
#    (gamma_m, gamma_p) for gamma_m in np.logspace(np.log10(0.0001), np.log10(0.1), 12)[:5].tolist()
#    for gamma_p in np.logspace(np.log10(0.0001), np.log10(0.1), 12)[:5].tolist()
#]

# Open all files in the directory dir_+"/models"
json_files = [f for f in os.listdir(dir_ + "/models") if f.endswith(".json")]
json_files = [json_files[i] for i in range(len(json_files)-1,-1,-1)]  # Shuffle files
models = []

print("Opening files...")
for file in tqdm(json_files):
    with open(os.path.join(dir_ + "/models", file), 'r') as f:
        model = json.load(f)
        models.append((file, model))
print("Done.")

print([key for key in models[0][1].keys()])  # Print keys of the first JSON file to see available data

Opening files...


100%|██████████| 720/720 [00:00<00:00, 752.27it/s]

Done.
['iterations', 'final_fidelity', 'final_purity', 'optimized_trainable_parameters', 'fidelity_each_timestep']


In [2]:
from concurrent.futures import ThreadPoolExecutor

def evaluate_model(params):
    filename, model = params
    if (os.path.exists(f"{dir_}/eval/{filename[:-5]}.json")):
        print(f"File {filename[:-5]}.json already exists. Skipping evaluation.")
        return

    N_qubits = int(re.search(r"Nqubits=(\d+)_", filename).group(1))
    N_meas = int(re.search(r"Nmeas=(\d+)_", filename).group(1))
    if gamma_p_global is None:
        gamma_p = float(re.search(r"gammap=([\d.]+)_", filename).group(1))
    else:
        gamma_p = gamma_p_global
    if gamma_m_global is None:
        gamma_m = float(re.search(r"gammam=([\d.]+)_", filename).group(1))
    else:
        gamma_m = gamma_m_global

    #if (gamma_m, gamma_p) not in accurate_pairs:
    #    print("Skipping evaluation of", filename, ", evaluating accurate pairs first.")
    #    return

    p = re.search(r"rhoe=([a-zA-Z0-9.]+)_", filename).group(1)
    if evaluation_state_type is None:
        evaluation_state = lambda key: state_types[p](key, N_qubits)
    else:
        filename = filename.replace("_rhoe="+p+"_", f"_rhoe={evaluation_state_type}_")
        evaluation_state = lambda key: state_types[evaluation_state_type](key, N_qubits)

    system_params = init_fgrape_protocol(
        jax.random.PRNGKey(0),
        N_qubits,
        N_meas,
        gamma_p,
        gamma_m,
    )

    eval_result = evaluate_on_longer_time(
        U_0 = evaluation_state,
        C_target = evaluation_state,
        system_params = system_params,
        optimized_trainable_parameters = model["optimized_trainable_parameters"],
        num_time_steps = evaluation_time_steps,
        evo_type = "density",
        goal = "fidelity",
        eval_batch_size = batch_size,# if (gamma_m, gamma_p) not in accurate_pairs else batch_size*4,
        mode = "lookup",
    )

    with open(f"{dir_}/eval/{filename[:-5]}.json", "w") as f:
        json.dump(FgResult_to_dict(eval_result), f)

    del eval_result  # Free memory
    del system_params  # Free memory
    print(f"Finished evaluation of {filename}.")

# Write Physical and evaluation parameters to a text file
with open(f"{dir_}/eval/evaluation_parameters.txt", "w") as f:
    f.write(f"Physical parameters:\n")
    f.write(f"gamma_p_global: {gamma_p_global}\n")
    f.write(f"gamma_m_global: {gamma_m_global}\n")
    f.write(f"evaluation_state_type: {evaluation_state_type}\n")
    f.write(f"\nEvaluation parameters:\n")
    f.write(f"evaluation_time_steps: {evaluation_time_steps}\n")
    f.write(f"batch_size: {batch_size}\n")

if N_parallel_threads == 1:
    for filename,model in tqdm(models):
        evaluate_model((filename, model))
else:
    with ThreadPoolExecutor(max_workers=N_parallel_threads) as executor:
        executor.map(evaluate_model, models)

# Play a sound when done
os.system('say "done."')

File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.004328761281083057_gammam=0.02848035868435799_rhot=bloch_rhoe=bloch_s=4.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.01519911082952933_gammam=0.01519911082952933_rhot=bloch_rhoe=bloch_s=0.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.008111308307896872_gammam=0.0003511191734215131_rhot=bloch_rhoe=bloch_s=2.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.05336699231206307_gammam=0.002310129700083158_rhot=bloch_rhoe=bloch_s=3.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.01519911082952933_gammam=0.1_rhot=bloch_rhoe=bloch_s=2.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap=0.1_gammam=0.0012328467394420659_rhot=bloch_rhoe=bloch_s=3.json already exists. Skipping evaluation.
File lut_t=3_l=3_w=111_Nqubits=3_Nmeas=3_gammap

0